<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_ResNet/test_sample_ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 1177 (delta 116), reused 156 (delta 60), pack-reused 938
Receiving objects: 100% (1177/1177), 8.51 MiB | 23.36 MiB/s, done.
Resolving deltas: 100% (687/687), done.


In [2]:
%cd PyTorch-Architectures/modeling_ResNet/

/content/PyTorch-Architectures/modeling_ResNet


In [4]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from model import ResNet, BasicBlock

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [5]:
# Hyperparameters

RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

NUM_FEATURES = 28 * 28
NUM_CLASSES = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = True

In [ ]:
# MNIST Dataset

train_dataset = datasets.MNIST(
    root="data",
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
test_dataset = datasets.MNIST(
    root="data",
    train=False,
    transform=transforms.ToTensor(),
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

In [7]:
# Checking the dataset
for images, labels in train_loader:
  print("Image Batch Dimensions: ", images.shape)
  print("Label Batch Dimensions: ", labels.shape)
  break

Image Batch Dimensions:  torch.Size([128, 1, 28, 28])
Label Batch Dimensions:  torch.Size([128])


In [8]:
def resnet34(NUM_CLASSES):
  model = ResNet(
      block=BasicBlock,
      layers=[3, 4, 6, 3],
      num_classes=NUM_CLASSES,
      grayscale=grayscale,
  )
  return model

In [9]:
torch.manual_seed(RANDOM_SEED)
model = resnet34(NUM_CLASSES)
model.to(device)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable Parameters: ", params)

Trainable Parameters:  21283530


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  for features, targets in data_loader:
    features = features.to(device)
    targets = targets.to(device)
    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    correct_pred += (predicted_labels == targets).sum()
    num_examples += targets.size(0)
  return correct_pred / num_examples * 100

start_time = time.time()
for epoch in range(NUM_EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    loss = F.cross_entropy(logits, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 50 == 0:
      print("Batch: %04d/%04d | Epoch: %04d/%04d | Cost: %.4f" % (idx, len(train_loader), epoch+1, NUM_EPOCHS, loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    print('Epoch: %04d/%04d | Train Acc: %.3f' % (epoch+1, NUM_EPOCHS, compute_accuracy(model, train_loader, device)))
  print('Epoch Elapsed Time: %.2f min' % ((time.time() - start_time) / 60))
print('Total Training Time: %.2f min' % ((time.time() - start_time) / 60))

Batch: 0000/0469 | Epoch: 0001/0010 | Cost: 2.5827
Batch: 0050/0469 | Epoch: 0001/0010 | Cost: 0.1939
Batch: 0100/0469 | Epoch: 0001/0010 | Cost: 0.2451
Batch: 0150/0469 | Epoch: 0001/0010 | Cost: 0.0646
Batch: 0200/0469 | Epoch: 0001/0010 | Cost: 0.0703
Batch: 0250/0469 | Epoch: 0001/0010 | Cost: 0.1670
Batch: 0300/0469 | Epoch: 0001/0010 | Cost: 0.0391
Batch: 0350/0469 | Epoch: 0001/0010 | Cost: 0.1063
Batch: 0400/0469 | Epoch: 0001/0010 | Cost: 0.1074
Batch: 0450/0469 | Epoch: 0001/0010 | Cost: 0.1380
Epoch: 0001/0010 | Train Acc: 98.162
Epoch Elapsed Time: 0.58 min
Batch: 0000/0469 | Epoch: 0002/0010 | Cost: 0.0825
Batch: 0050/0469 | Epoch: 0002/0010 | Cost: 0.0354
Batch: 0100/0469 | Epoch: 0002/0010 | Cost: 0.0128
Batch: 0150/0469 | Epoch: 0002/0010 | Cost: 0.0363
Batch: 0200/0469 | Epoch: 0002/0010 | Cost: 0.0160
Batch: 0250/0469 | Epoch: 0002/0010 | Cost: 0.0192
Batch: 0300/0469 | Epoch: 0002/0010 | Cost: 0.0859
Batch: 0350/0469 | Epoch: 0002/0010 | Cost: 0.1351
Batch: 0400/0469

In [12]:
model.eval()
with torch.set_grad_enabled(False):
  print('Test Accuracy: %.2f' % (compute_accuracy(model, test_loader, device)))

Test Accuracy: 98.93
